# Tutorial, Part I: Candidate Extraction

In this example, we'll be writing an application to extract **person-age relationships** from homemade tables, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _person-age relation mentions_ as either true or false.  To do this, we first need a set of such candidates- in this notebook, we'll use `Snorkel` utilities to extract person candidates.

## Loading the Corpus

First, we will load and pre-process the corpus, storing it for convenience in a `Corpus` object

### Configuring a table parser

We'll start by defining an 'HTMLTableParser' class to read HTML tables.

In [5]:
%load_ext autoreload
%autoreload 2

from ddlite_parser import HTMLTableParser
html_parser = HTMLTableParser(
    path='data/Tables_DevelopmentSet.xml')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Selecting a cell parser

Next, we'll use an NLP preprocessing tool to split the `Document` objects into sentences, tokens, and provide annotations--part-of-speech tags, dependency parse structure, lemmatized word forms, etc.--for these sentences.  Here we use the default `SentenceParser` class.

In [6]:
from ddlite_parser import SentenceParser
sent_parser = SentenceParser()

### Pre-processing & loading the corpus

Finally, we'll put this all together using a `Corpus` object, which will execute the parsers and store the results as an iterator:

In [ ]:
from ddlite_parser import Corpus
%time corpus = Corpus(html_parser, sent_parser)

Parsing documents...
> /Users/bradenhancock/snorkel/ddlite_parser.py(235)__init__()
-> print "Parsing sentences..."
(Pdb) self._docs_by_id
{<built-in function id>: Document(id=<built-in function id>, file='Tables_DevelopmentSet.xml', text=<table>\n<tr>\n<th>Person</th>\n<th>Age</th>\n<th>Birthdate</th>\n</tr>\n<tr>\n<th>Alice</th>\n<td>45</td>\n<td>Mar. 15, 1971</td>\n</tr>\n<tr>\n<th>Bob</th>\n<td>17</td>\n<td>Jan. 1, 1999</td>\n</tr>\n<tr>\n<th>Charlie</th>\n<td>80</td>\n<td>July 13, 1936</td>\n</tr>\n<caption>\n    Table 1: Ages and birthdates of the world's most important people.\n  </caption>\n</table>, attribs=None)}
(Pdb) self._docs_by_id[0]
*** KeyError: 0
(Pdb) self._docs_by_id.values()
[Document(id=<built-in function id>, file='Tables_DevelopmentSet.xml', text=<table>\n<tr>\n<th>Person</th>\n<th>Age</th>\n<th>Birthdate</th>\n</tr>\n<tr>\n<th>Alice</th>\n<td>45</td>\n<td>Mar. 15, 1971</td>\n</tr>\n<tr>\n<th>Bob</th>\n<td>17</td>\n<td>Jan. 1, 1999</td>\n</tr>\n<tr>\n<th>Charlie

In [ ]:
doc  = corpus.get_docs()[0]
print doc

In [ ]:
sent = corpus.get_sentences_in(doc.id)[0]
print sent

## Writing a basic candidate extractor

Next, we'll write a basic function to extract **candidate disease mentions** from the corpus.  For this first attempt, we'll just write a function that checks for matches against a list (or _"dictionary"_) of disease phrases, constructed using some pre-compiled ontologies ([UMLS](https://www.nlm.nih.gov/research/umls/), [ORDO](http://www.orphadata.org/cgi-bin/inc/ordo_orphanet.inc.php), [DOID](http://www.obofoundry.org/ontology/doid.html), [NCBI Diseases](http://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/); see `tutorial/data/diseases.py`).

We'll do this using a `CandidateSpace` object--which defines the basic candidates we consider, in this case n-grams up to a certain length--and a `Matcher` object, which filters this candidate space down.

In [ ]:
from load_dictionaries import load_disease_dictionary
from ddlite_candidates import Ngrams
from ddlite_matchers import DictionaryMatch

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

# Define a candidate space
ngrams = Ngrams(n_max=3)

# Define a matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Note that we set `longest_match_only=False`, which means that we _will_ consider subsequences of phrases that match our dictionary.

The `Ngrams` operator is applied over our `Sentence` objects and returns `Ngram` objects, and the `Matcher` then filters these, so we apply our operators over the sentences in the corpus, storing the results in a `Candidates` object for convenience:

In [ ]:
from ddlite_candidates import Candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.get_candidates()[:5]

## Evaluating our candidate recall on gold annotations

Next, we'll test our _candidate recall_--in other words, how many of the true disease mentions we picked up in our candidate set--using the gold annotations in our dataset.

The XML documents that we loaded using the `XMLDocParser` also contained annotations (this is why we kept the full xml tree using `keep_xml_tree=True`).  We'll load these annotations and map them to `Ngram` objects over our parsed sentences, that way we can easily compare our extracted candidate set with the gold annotations.  The code is fairly simple (see `tutorial/util.py`); note that we filter to only keep _disease_ annotations, and that the candidates should be uniquely identified by their `id` attribute:

In [ ]:
from utils import collect_pubtator_annotations
gold = []
for doc, sents in corpus:
    gold += [a for a in collect_pubtator_annotations(doc, sents) if a.metadata['type'] == 'Disease']
gold = frozenset(gold)

Now, we have a set of gold annotations of the same type as our candidates (`Ngram`), and can use set operations (where candidate objects are hashed by their `id` attribute), e.g.:

In [ ]:
len(gold.intersection(c.get_candidates()))

For convenience, we'll use a basic helper method of the `Candidates` object:

In [ ]:
c.gold_stats(gold)

We note that our focus in this stage is on **acheiving high candidate recall, without considering an impractically large candidate set**.  Our main focus after this stage will be on training a classifier to select which candidates are true; this will raise precision while hopefully keeping recall high.  _Note however that candidate recall is an upper bound for the recall of this classifier!_

So, we have some work to do.

## Using the `Viewer` to inspect data

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

To start, we'll assemble a random set of all the sentences where there are gold annotations _not in our candidate set_, i.e. where we missed something, and then inspect these in the `Viewer`:

In [ ]:
from collections import defaultdict
from random import shuffle

# Index the gold annotations by sentence id
gold_by_sid = defaultdict(list)
for g in gold:
    gold_by_sid[g.sent_id].append(g)

# Get sentences
view_sents = [s for s in corpus.get_sentences() if len(c.get_candidates_in(s.id)) < len(gold_by_sid[s.id])]
shuffle(view_sents)
view_sents = view_sents[:50]

Now, we instantiate and render the `Viewer` object; note we're being a bit sloppy, passing in _all_ the candidates and gold labels, but the `Viewer` object will take care of indexing them by sentence, and will only render the sentences we pass in:

In [ ]:
from ddlite_viewer import SentenceNgramViewer
sv = SentenceNgramViewer(view_sents, c.get_candidates(), gold=gold)
sv.render(n_per_page=3, height=225)

## Composing a better candidate extractor

Let's try to increase our candidate recall using more of the `Matcher` operators and their functionalities.  First, let's turn on **Porter stemming** in our dictionary matcher; Porter stemming is an aggressive rules-based method for normalizing word endings.

In [ ]:
# Define a new matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter')

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Next, note that *`Matcher` objects are compositional*. Observing in the `Viewer` that we are missing all of the acronyms, let's start with the `Union` operator, to integrate a dictionary for this:

In [ ]:
from ddlite_matchers import Union
from load_dictionaries import load_acronym_dictionary

# Load the disease phrase dictionary
acronyms = load_acronym_dictionary()
print "Loaded %s acronyms!" % len(acronyms)

# Define a new matcher
matcher = Union(
    DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Next, we try using the `Concat` and `RegexMatch` operators to find candidate mentions composed of an _adjective followed by a term matching our diseases dictionary_.  Note in particular that we set `left_required=False` so that exact matches to our dictionary (with no adjective prepended) will still work:

In [ ]:
from ddlite_matchers import Concat, RegexMatchEach
matcher = Union(
    Concat(
        RegexMatchEach(rgx=r'JJ*', attrib='poses'),
        DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'),
        left_required=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

### Running candidate extraction in parallel

Note that **the candidate extraction procedure can be parallelized across multiple cores using the `parallelism=N`** optional argument to the `Candidates` object.

### More coming here...

We've increased the candidate recall (on the development set) by 7.3% using some simple compositional `Matcher` operators.  We'll be adding more here soon!

## Connecting with the rest of the `DDLite` workflow

We are in the process of a big code refactor!  For now, to connect the candidates derived as in above to the rest of the `DDLite` workflow, create a `CandidateExtractor` class as follows:

In [ ]:
from ddlite_matchers import CandidateExtractor
ce = CandidateExtractor(ngrams, matcher)

This object can now be used in place of the candidate extractors in the other tutorials!